# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the data analysis and modeling tasks to the Data Analyst in order to identify trends and predict market movements for AAPL.

Action: Delegate work to co-worker
Action Input: 
{
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL, using statistical modeling and machine learning to identify trends and predict market movements.",
    "context": "AAPL is the selected stock for analysis. The goal is to provide insights and alerts about significant market opportunities or threats for AAPL."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL and analyze it using statistical modeling and machine learning to identif

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science Originals The 360 Newsletters Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Finance My Portfolio News Latest Stock Market Originals The Morning Brief Economies Premium News Earnings Tech Housing Crypto Biden Economy Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Communication 

Final Answer: 
The Data Analyst has been delegated the task of continuously monitoring and analyzing market data for AAPL, using statistical modeling and machine learning to identify trends and predict market movements. The goal is to provide insights and alerts about significant market opportunities or threats for AAPL.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: The Data Analyst has been delegated the task of continuously monitoring and analyzing market data for AAPL, using statistical modeling and machine learning to identify trends and predict market movements. The goal is to provide insights and alerts about significant market opportunities or threats for AAPL.
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Develop and refine trading strategies based on the insights from the Data Analyst and user-defined risk tolerance (Medium). Consider trading preferences (Day Trading).


> Entering new CrewAgentExecutor chain...
I need to ensure that the trading stra

I need to delegate the analysis of the approved trading strategies to determine the best execution methods for AAPL to the Trading Strategy Developer. They will have the expertise to evaluate the strategies and provide detailed execution plans.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Analyze approved trading strategies for AAPL and determine the best execution methods", "context": "After analyzing the insights provided by various analysts on AAPL stock, the Trading Strategy Developer needs to evaluate the Momentum, Breakout, Swing, and Day Trading strategies to provide detailed execution plans considering current market conditions and optimal pricing."}


> Entering new CrewAgentExecutor chain...
I need to gather information on the Momentum, Breakout, Swing, and Day Trading strategies for AAPL stock in order to evaluate and determine the best execution methods.

Action: Search the internet
Action Input: {"search_query": "AAP

Thought: 
I have gathered valuable information on Momentum, Breakout, and Swing trading strategies for AAPL stock. I should now search for Day Trading strategies.

Action: Search the internet
Action Input: {"search_query": "AAPL stock Day Trading strategy"} 


Search results: Title: Stock Trading in AAPL: 27 Things Traders Should Know About
Link: https://www.quantifiedstrategies.com/stock-trading-aapl/
Snippet: No, AAPL stocks cannot be traded 24 hours a day. AAPL stocks can be traded during regular trading hours, which are usually 9:30 am to 4:00 pm ET. However, the ...
---
Title: How to Analyze and Day Trade Apple Stock (AAPL) Successfully
Link: https://realtrading.com/trading-blog/how-to-day-trade-apple-stock/
Snippet: There are several strategies to day trade Apple shares. First, the most common strategy is known as trend moving. This is a strategy that ...
---
Title: Apple Stock: 3 Simple Day Trading Strategies - Tradingsim
Link: https://app.tradingsim.com/blog/apple-stock-3-simpl

I need to gather detailed information from each team member to evaluate the risks associated with the proposed trading strategies for AAPL and provide mitigation strategies.

Action: Delegate work to co-worker
Action Input: {"coworker": "Risk Advisor", "task": "Evaluate risks associated with the proposed trading strategies for AAPL and provide mitigation strategies", "context": "I need you to thoroughly analyze the risks involved in the Momentum Trading Strategy, Breakout Trading Strategy, Swing Trading Strategy, and Day Trading Strategy for AAPL. Consider market trends, resistance levels, stop-loss orders, support levels, and risk management strategies. Provide detailed analysis and suggest mitigation strategies to minimize potential risks."}

> Entering new CrewAgentExecutor chain...
I need to gather detailed information on the risks associated with the proposed trading strategies for AAPL and provide mitigation strategies to minimize these risks. I should focus on market trends, res

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

As the Crew Manager, I have thoroughly evaluated the risks associated with the proposed trading strategies and execution plans for AAPL. After analyzing the Momentum Trading Strategy, Breakout Trading Strategy, Swing Trading Strategy, and Day Trading Strategy, I have identified several potential risks and have suggested mitigation strategies to minimize these risks.

1. Momentum Trading Strategy:
- Potential Risks: Buying AAPL stock based on strong upward momentum may lead to entering the market at a peak, resulting in losses if the trend reverses quickly. Selling AAPL stock when signs of reversal occur may result in missed opportunities for further gains.
- Mitigation Strategies: Implementing tight stop-loss orders to protect profits and limit losses. Continuously monitoring market trends and adjusting trading positions accordingly to avoid significant losses.

2. Breakout Trading Strategy:
- Potential Risks: Buying AAPL stock when it breaks out of a key resistance level may result in false breakouts, leading to losses. Failing to maintain momentum and capture price movements following significant market opportunities may result in missed profits.
- Mitigation Strategies: Implementing tight stop-loss orders to manage risks and protect profits. Conducting thorough analysis to confirm breakout signals and avoid false breakouts.

3. Swing Trading Strategy:
- Potential Risks: Holding AAPL stock for a few days to weeks may expose the trader to market fluctuations and volatility, resulting in potential losses. Buying at support levels and selling at resistance levels may not always yield profitable trades.
- Mitigation Strategies: Implementing trailing stop orders to secure profits and minimize losses in volatile market conditions. Conducting thorough analysis and using technical indicators to confirm support and resistance levels.

4. Day Trading Strategy:
- Potential Risks: Buying and selling AAPL stock within the same trading day based on intraday market trends may result in emotional trading decisions and impulsive trades. Failing to adhere to strict risk management strategies may lead to consistent losses.
- Mitigation Strategies: Monitoring market data in real-time and executing trades based on a predefined plan. Implementing strict risk management strategies and adhering to trading rules to achieve consistent profitability in day trading activities.

Overall, by considering these risks and implementing the suggested mitigation strategies, traders can effectively navigate the market while trading AAPL stock based on the insights from the Data Analyst and their defined risk tolerance.